<a href="https://colab.research.google.com/github/Pavan-Gandham/ML-AI/blob/master/flipkartGrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv('/content/generated5.csv')
for col in df.columns:
    if 'Unnamed' in col:
        del df[col]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27472 entries, 0 to 27471
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           27472 non-null  int64  
 1   user_name         27472 non-null  object 
 2   preferences       27472 non-null  object 
 3   location          27472 non-null  object 
 4   product_id        27472 non-null  int64  
 5   description       27472 non-null  object 
 6   price             27472 non-null  float64
 7   category          27472 non-null  object 
 8   popularity        27472 non-null  float64
 9   ratings           27472 non-null  object 
 10  interaction_type  27472 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 2.3+ MB


In [ ]:
df.head()

,user_id,user_name,preferences,location,product_id,description,price,category,popularity,ratings,interaction_type
0,1,User1,"Electronics,Home",City8,2913,Gadgets technology devices devices gadgets.,186.14,Electronics,0.27,"3021:3,143:4,758:5,4883:4,3276:4,739:4,2983:3,...",AddToCart
1,1,User1,"Electronics,Home",City8,4297,Equipment fitness equipment exercise equipment...,611.53,Sports,0.61,"1304:5,1954:3,2608:2,839:3,3228:1,2419:3,3283:...",Buy
2,1,User1,"Electronics,Home",City8,2425,Devices electronics technology technology devi...,304.85,Electronics,0.44,"1606:1,1576:5,1832:5,950:3,2721:2,3023:3,4695:...",View
3,1,User1,"Electronics,Home",City8,3376,Exercise fitness sports equipment fitness fitn...,573.30,Sports,0.98,"540:5,1402:3,4551:5,1596:4,2696:4,3559:3,1333:...",AddToCart
4,1,User1,"Electronics,Home",City8,1202,Fitness exercise equipment sports sports sport...,905.27,Sports,0.54,"3973:1,1677:3,2688:4,3186:2,3592:2,2370:2,1519...",Buy


In [ ]:
data = df['interaction_type']
interaction_mapping = {
    "Buy": 1,
    "View": 2,
    "AddToCart": 3
}
encoded_value = [interaction_mapping[i] for i in data]
# encoded_value[0:5]
df['interaction_type'] = encoded_value
df.head()

,user_id,user_name,preferences,location,product_id,description,price,category,popularity,ratings,interaction_type
0,1,User1,"Electronics,Home",City8,2913,Gadgets technology devices devices gadgets.,186.14,Electronics,0.27,"3021:3,143:4,758:5,4883:4,3276:4,739:4,2983:3,...",2
1,1,User1,"Electronics,Home",City8,4297,Equipment fitness equipment exercise equipment...,611.53,Sports,0.61,"1304:5,1954:3,2608:2,839:3,3228:1,2419:3,3283:...",0
2,1,User1,"Electronics,Home",City8,2425,Devices electronics technology technology devi...,304.85,Electronics,0.44,"1606:1,1576:5,1832:5,950:3,2721:2,3023:3,4695:...",1
3,1,User1,"Electronics,Home",City8,3376,Exercise fitness sports equipment fitness fitn...,573.30,Sports,0.98,"540:5,1402:3,4551:5,1596:4,2696:4,3559:3,1333:...",2
4,1,User1,"Electronics,Home",City8,1202,Fitness exercise equipment sports sports sport...,905.27,Sports,0.54,"3973:1,1677:3,2688:4,3186:2,3592:2,2370:2,1519...",0


In [ ]:
#Creating the interaction matrix of products and users based on ratings and replacing NaN value with 0

# pivot table with interactions as value
features_to_include = ['preferences', 'location', 'price', 'category', 'popularity', 'ratings', 'interaction_type']
interaction_ratings_matrix = df.pivot_table(index = ['user_id'], columns =['product_id'], values = ['interaction_type']).fillna(0)
print('Shape of interaction_ratings_matrix: ', interaction_ratings_matrix.shape)

#Finding the number of non-zero entries in the interaction matrix
given_num_of_ratings = np.count_nonzero(interaction_ratings_matrix)
print('given_num_of_ratings = ', given_num_of_ratings)

#Finding the possible number of ratings as per the number of users and products
possible_num_of_ratings = interaction_ratings_matrix.shape[0] * interaction_ratings_matrix.shape[1]
print('possible_num_of_ratings = ', possible_num_of_ratings)

interaction_ratings_matrix.head()

Shape of interaction_ratings_matrix:  (1000, 4974)
given_num_of_ratings =  18309
possible_num_of_ratings =  4974000


interaction_type                                               ...  \
product_id             1    2    3    4    5    6    7    8    9    10    ...   
user_id                                                                   ...   
1                       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
2                       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
3                       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
4                       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
5                       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   

                                                              
product_id 4990 4991 4992 4993 4994 4995 4996 4997 4998 4999  
user_id                                                       
1           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
5           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 4974 columns]

In [ ]:
data = df['preferences']

preferences_mapping = {
    "Home": 0,
    "Books": 1,
    "Electronics": 2,
    "Fashion" : 3,
    "Sports" : 4
}

# Function to encode preferences as a comma-separated string
def encode_preferences(preferences):
    if pd.notna(preferences):
        preference_list = preferences.split(',')
        encoded_preferences = ','.join([str(preferences_mapping[val]) for val in preference_list if val in preferences_mapping])
        return encoded_preferences
    else:
        return ''

# Apply the encoding function to the 'preferences' column
df['preferences'] = df['preferences'].apply(lambda x: encode_preferences(x))


In [ ]:
df.head()

,user_id,user_name,preferences,location,product_id,description,price,category,popularity,ratings,interaction_type
0,1,User1,"2,0",City8,2913,Gadgets technology devices devices gadgets.,186.14,Electronics,0.27,"3021:3,143:4,758:5,4883:4,3276:4,739:4,2983:3,...",2
1,1,User1,"2,0",City8,4297,Equipment fitness equipment exercise equipment...,611.53,Sports,0.61,"1304:5,1954:3,2608:2,839:3,3228:1,2419:3,3283:...",0
2,1,User1,"2,0",City8,2425,Devices electronics technology technology devi...,304.85,Electronics,0.44,"1606:1,1576:5,1832:5,950:3,2721:2,3023:3,4695:...",1
3,1,User1,"2,0",City8,3376,Exercise fitness sports equipment fitness fitn...,573.30,Sports,0.98,"540:5,1402:3,4551:5,1596:4,2696:4,3559:3,1333:...",2
4,1,User1,"2,0",City8,1202,Fitness exercise equipment sports sports sport...,905.27,Sports,0.54,"3973:1,1677:3,2688:4,3186:2,3592:2,2370:2,1519...",0


In [ ]:
pref_df =  df.groupby('user_id')['preferences'].apply(lambda x: ','.join(set(x))).reset_index()

In [ ]:
preference_lists = []
for p in pref_df['preferences']:
  curr_pref = [int(i) for i in p.split(',')]
  preference_lists.append(curr_pref)
user_id = [i for i in range(1, 1001)]
preferences = ['0', '1', '2', '3', '4']

In [ ]:
data = {'user_id': user_id, 'preference_list': preference_lists}
pre_df = pd.DataFrame(data)

# Create a binary preference DataFrame using pivot_table
pref_pivot_table = pre_df.explode('preference_list').pivot_table(
    index='user_id',
    columns='preference_list',
    values='preference_list',
    aggfunc=lambda x: 1,
    fill_value=0
)

pref_pivot_table.head()


preference_list,0,1,2,3,4
user_id,,,,,
1,1,0,1,0,0
2,0,0,1,0,0
3,0,0,1,0,0
4,0,1,1,0,0
5,0,0,1,1,1


In [ ]:
selected_columns = ['user_id', 'ratings']
new_df = df[selected_columns]
new_df.head()

,user_id,ratings
0,1,"3021:3,143:4,758:5,4883:4,3276:4,739:4,2983:3,..."
1,1,"1304:5,1954:3,2608:2,839:3,3228:1,2419:3,3283:..."
2,1,"1606:1,1576:5,1832:5,950:3,2721:2,3023:3,4695:..."
3,1,"540:5,1402:3,4551:5,1596:4,2696:4,3559:3,1333:..."
4,1,"3973:1,1677:3,2688:4,3186:2,3592:2,2370:2,1519..."


In [ ]:
grouped_df = df.groupby('user_id')['ratings'].apply(','.join).reset_index()
grouped_df.head()

,user_id,ratings
0,1,"3021:3,143:4,758:5,4883:4,3276:4,739:4,2983:3,..."
1,2,"2158:3,4801:5,678:2,919:3,234:1,3761:4,2714:2,..."
2,3,"3148:2,950:1,3362:4,1781:1,3936:1,1077:2,3349:..."
3,4,"2394:3,3634:2,735:4,3226:5,2751:1,761:4,4981:1..."
4,5,"1222:3,4115:2,2308:2,750:1,3223:5,3082:4,1942:..."


In [ ]:
# pivot table with popularity
products = [] # stores list of products in order of
ratings = []
product_ratings = grouped_df['ratings']
product_ratings



0      3021:3,143:4,758:5,4883:4,3276:4,739:4,2983:3,...
1      2158:3,4801:5,678:2,919:3,234:1,3761:4,2714:2,...
2      3148:2,950:1,3362:4,1781:1,3936:1,1077:2,3349:...
3      2394:3,3634:2,735:4,3226:5,2751:1,761:4,4981:1...
4      1222:3,4115:2,2308:2,750:1,3223:5,3082:4,1942:...
                             ...                        
995    3223:4,602:2,3390:1,264:2,652:3,4102:4,2281:1,...
996    3375:3,3245:3,1600:2,2932:4,3842:1,1044:3,2187...
997    4030:2,3008:3,2312:4,1849:3,2533:5,3707:5,4178...
998    3789:1,3498:3,4816:3,3582:2,1339:2,3609:5,1454...
999    1410:2,1089:5,4857:2,1098:4,2411:4,1850:3,1391...
Name: ratings, Length: 1000, dtype: object

In [ ]:
products = [] # stores list of products in order of
ratings = []
for rating_item in product_ratings:
  p = []
  r = []
  ind_rat = rating_item.split(',')
  for rat in ind_rat:
    p_id, rat = rat.split(":")[0],rat.split(":")[1]
    p.append(p_id)
    r.append(rat)
  products.append(p)
  ratings.append(r)
# len(products)


In [ ]:
user_ids = [i for i in range(1, 1001)]


In [ ]:
data = {'user_id': user_ids, 'product_ids': products, 'ratings': ratings}
df = pd.DataFrame(data)

# Create a list of dictionaries for product-rating pairs
product_rating_dicts = []
for _, row in df.iterrows():
    user_id = row['user_id']
    for product_id, rating in zip(row['product_ids'], row['ratings']):
        product_rating_dicts.append({'user_id': user_id, 'product_id': product_id, 'rating': rating})

# Create a DataFrame from the list of dictionaries
# print(product_rating_dicts)
ratings_pivot_df = pd.DataFrame(product_rating_dicts)
ratings_pivot_df = ratings_pivot_df.drop_duplicates(subset=['user_id', 'product_id'])
# Pivot the DataFrame
ratings_pivot_df = ratings_pivot_df.pivot(index='user_id', columns='product_id', values='rating').fillna(0)

ratings_pivot_df.head()


product_id,1,10,100,1000,1001,1002,1003,1004,1005,1006,...,990,991,992,993,994,995,996,997,998,999
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,1,0,...,0,4,0,0,0,0,0,0,1,2
2,0,0,2,0,0,0,0,0,0,0,...,0,0,0,1,0,3,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
5,0,0,3,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# defining a function to get similar users
def similar_users(user_index, interactions_matrix):
    similarity = []
    for user in range(0, interactions_matrix.shape[0]): #  .shape[0] gives number of rows

        #finding cosine similarity between the user_id and each user
        sim = cosine_similarity([interactions_matrix.iloc[user_index]], [interactions_matrix.iloc[user]])

        #Appending the user and the corresponding similarity score with user_id as a tuple
        similarity.append((user,sim))

    similarity.sort(key=lambda x: x[1], reverse=True)
    most_similar_users = [tup[0] for tup in similarity] #Extract the user from each tuple in the sorted list
    similarity_score = [tup[1] for tup in similarity] ##Extracting the similarity score from each tuple in the sorted list

    #Remove the original user and its similarity score and keep only other similar users
    most_similar_users.remove(user_index)
    similarity_score.remove(similarity_score[0])

    return most_similar_users, similarity_score

In [ ]:
similar = similar_users(999, final_ratings_matrix)[0][0:10]
# similar = similar_users(1521, final_ratings_matrix)[0][0:10]
similar

[555, 962, 431, 2, 693, 216, 169, 381, 175, 774]

In [ ]:
# defining the recommendations function to get recommendations by using the similar users' preferences
def recommendations(user_index, num_of_products, interactions_matrix):

    #Saving similar users using the function similar_users defined above
    most_similar_users = similar_users(user_index, interactions_matrix)[0]

    #Finding product IDs with which the user_id has interacted
    prod_ids = set(list(interactions_matrix.columns[np.where(interactions_matrix.iloc[user_index] > 0)]))
    recommendations = []

    observed_interactions = prod_ids.copy()
    for similar_user in most_similar_users:
        if len(recommendations) < num_of_products:

            #Finding 'n' products which have been rated by similar users but not by the user_id
            similar_user_prod_ids = set(list(interactions_matrix.columns[np.where(interactions_matrix.iloc[similar_user] > 0)]))
            recommendations.extend(list(similar_user_prod_ids.difference(observed_interactions)))
            observed_interactions = observed_interactions.union(similar_user_prod_ids)
        else:
            break

    return recommendations[:num_of_products]

In [ ]:
all_pivot_tables = [ratings_pivot_df, interaction_ratings_matrix]
user_item_matrix = pd.concat(all_pivot_tables).pivot_table(index='user_id', columns='product_id', values='Interaction', fill_value=0)
weights = [0.7, 0.3]
weighted_user_item_matrix = np.zeros_like(user_item_matrix)
for i, pivot_table in enumerate(all_pivot_tables):
    weighted_user_item_matrix += pivot_table.pivot_table(index='User', columns='Item', values='Interaction', fill_value=0) * weights[i]
weighted_user_item_matrix

product_id,1,10,100,1000,1001,1002,1003,1004,1005,1006,...,990,991,992,993,994,995,996,997,998,999
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,1,0,...,0,4,0,0,0,0,0,0,1,2
2,0,0,2,0,0,0,0,0,0,0,...,0,0,0,1,0,3,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
5,0,0,3,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
recommendations(1,5,weighted_user_item_matrix)

3627
3757
3702
1347
2030


In [ ]:
def calculate_precision_recall(true_relevant_items, recommended_items):
    # Calculate the intersection of true relevant items and recommended items
    intersection = set(true_relevant_items) & set(recommended_items)

    # Calculate Precision and Recall
    precision = len(intersection) / len(recommended_items) if len(recommended_items) > 0 else 0
    recall = len(intersection) / len(true_relevant_items) if len(true_relevant_items) > 0 else 0

    return precision, recall

precision, recall = calculate_precision_recall(true_relevant_items, recommended_items)
print("Precision:", precision)
print("Recall:", recall)


Precision: 0.8
Recall: 0.8
